# The Elite Ant System Algorithm For The Travelling Salesman Problem

Algorithm from the Book **Ant Colony Optimization by Marco Dorigo and Thomas Stützle**

In [2]:
#----------------------------------------
# Chia E. Tungom
# chemago99@yahoo.com
# July 30th 2020
# Shenzhen China
#----------------------------------------

In [3]:
# import libraries

import numpy as np
import math
import random

# Roulette Wheel for Probability

In [68]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    """ pi: list with probability values
        return roulette proportions"""
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    """ ppi: list of roulette proportions
        Pop: list of corresponding item   [item1, ... , itemN] """
    
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

# Calculate Route Cost and Rank Ants

In [88]:
# calculate cost of each route
def Cost(Pop, graph):
    """ Pop: List of All Items in the e.g [[item1], ... , [itemN]]
    graph: cost matrix or graph 
    returns a cost vector for corresponding item """
    
    Cvector = []
    
    for i in range(len(Pop)):
        route = Pop[i]
        cost = 0
        
        for j in range(len(route)-1):
            cost += graph[route[j]][route[j+1]]
        cost += graph[route[0]][route[len(route)-1]]   
        Cvector.append(cost)
        
    return Cvector

# Sort Chromosomes from Best to Worst
def Rank(Pop, graph):
    """ returns sorted Ants """
    cost = Cost(Pop, graph)
    ans = [ x for _,x in sorted(zip(cost, Pop))]
    #cost = [ _ for _,x in sorted(zip(cost, Pop))]  # need to use the true cost
    return ans

# fitness function 1
def F1(sortedpop):
    bb= 0.3
    
    F = []   
    for i in range(1,len(sortedpop)+1):
        F.append( bb * ( (1-bb)**(i-1) ) )
        
    return F

# fitness function 1
def F2(sortedpop):
    n = len(sortedpop)
    
    F = []
    for i in range(1,n+1):
        F.append( (n-i+1)/n )
        
    return F

# Initialize Pheromone 

In [89]:
# AS initialization
def TSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    pher = [[cost for i in range(n)] for j in range(n)]
    
    return pher

# EAS initialization
def EASTSPher(graph, e, m, rho):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    
    numer = e + m
    den = rho*cost
    pher = [[numer/den for i in range(n)] for j in range(n)]
    
    return pher

# Calculate Transition Probability

In [90]:
# Transition Probability
def TransitionP(connections, alpha, beta):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]**alpha)*(i[1]**beta) for i in connections])    # sum heuristic and pheromone info
    prob = [( (((i**alpha)*(j**beta))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

def Disconnect(visited, connections):
    """ Disconnects already visited hobs
    visited: already visited hobs
    connections: connection vector with 0: no connection x: length of connection"""
    
    for i in visited:
        connections[i] = 0
        
    return connections

# Find Path 

In [91]:
def FindPath(phermatrix, graph, takeoff, alpha = 2, beta = 2):
    
    path = []
    path.append(takeoff)
    
    while len(path) != len(graph):
        Cons = graph[takeoff].copy()
        tabu = path.copy()
        Cons = Disconnect(tabu, Cons)
        HP = DPH(phermatrix, graph, Cons, takeoff)
        prob = TransitionP(HP, alpha, beta)
        takeoff = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob])
        
        path.append(takeoff)
 
    return path

# Get Elite Route and Connections

In [92]:
def Elite(Elite, ant, graph):
    
    if Cost([ant], graph)[0] < Cost([Elite], graph)[0]:
        return ant
    else:
        return Elite

def connections(L):
    """L is a list
    one way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
    
    cons.append((L[-1], L[0]))
    
    return cons

def connectionsE(L):
    """L is a list
    Two way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
        cons.append( (L[i], L[i-1]) )
    
    cons.append((L[-1], L[0]))
    cons.append((L[0], L[1]))
    
    return cons

# Update Pheromone 

In [93]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix

# Rank Based Ant System
def RBASUpPheromone(route, EliteRoute, matrix, e, w, r):  # problematic when multiplied by (w-r)
    """route : route taken
    EliteRoute: Best Route found thus far
    matrix: pheromone matrix
    e: weighting factor for elite pheromone"""
    
    Tbs = connectionsE(EliteRoute)
    n = len(Tbs)
    bb = 0.3
    
    bstau = 1/Cost([EliteRoute], matrix)[0]
    #tau = 1/Cost([route], matrix)[0]
    #tau = 1
    tau = bb * ( (1-bb)**(r-1) )
    #tau = (n-r+1)/n
    
    #print(bstau, tau)
    
    for i in range(len(route)-1):
        
        if (route[i], route[i+1]) in Tbs:
            matrix[route[i]][route[i+1]] = matrix[route[i]][route[i+1]] + (w-r)*tau + (e*bstau)
            matrix[route[i+1]][route[i]] = matrix[route[i+1]][route[i]] + (w-r)*tau + (e*bstau)
        else:
            matrix[route[i]][route[i+1]] = matrix[route[i]][route[i+1]] + (w-r)*tau
            matrix[route[i+1]][route[i]] = matrix[route[i+1]][route[i]] + (w-r)*tau
    
    # add pheromone to last and first route for TSP
    if (route[-1], route[0]) in Tbs:
        matrix[route[-1]][route[0]] = matrix[route[-1]][route[0]] + (w-r)*tau + (e*bstau)
        matrix[route[0]][route[-1]] = matrix[route[0]][route[-1]] + (w-r)*tau + (e*bstau)
    else:
        matrix[route[-1]][route[0]] = matrix[route[-1]][route[0]] + (w-r)*tau
        matrix[route[0]][route[-1]] = matrix[route[0]][route[-1]] + (w-r)*tau

    return matrix

# Elite based Ant System
def EASUpPheromone(route, EliteRoute, matrix, e):
    """route : route taken
    EliteRoute: Best Route found thus far
    matrix: pheromone matrix
    e: weighting factor for elite pheromone
    r: rank of ant
    w: number of top ants used """
    
    Tbs = connectionsE(EliteRoute)
    
    bstau = 1/Cost([EliteRoute], matrix)[0]
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        if (route[i], route[i+1]) in Tbs:
            matrix[route[i]][route[i+1]] = matrix[route[i]][route[i+1]] + tau + (e*bstau)
            matrix[route[i+1]][route[i]] = matrix[route[i+1]][route[i]] + tau + (e*bstau)
        else:
            matrix[route[i]][route[i+1]] += tau
            matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    if (route[-1], route[0]) in Tbs:
        matrix[route[-1]][route[0]] = matrix[route[-1]][route[0]] + tau + (e*bstau)
        matrix[route[0]][route[-1]] = matrix[route[0]][route[-1]] + tau + (e*bstau)
    else:
        matrix[route[-1]][route[0]] += tau
        matrix[route[0]][route[-1]] += tau

    return matrix

# Evaporate Pheromone 

In [94]:
def EvPheromone(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = math.exp(-5)
    return matrix

def EvPheromoneNeg(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            matrix[i][j] = (1-rho)*matrix[i][j]
        
    return matrix

# Generate a Toy Symmetric TSP Problem 

In [95]:
# A random graph
def Graph(dim, bounds):
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([random.randint(bounds[0], bounds[1])]))  # route length
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Ant System Algorithm

In [100]:
def AS(Ngraph, ants, alpha = 1, beta = 3, rho = 0.5, e = 100, w = 5 , runs = 50, Pher = None):
    
    if Pher == None:
        #Pher = TSPher(Ngraph)
        Pher = EASTSPher(Ngraph, e, ants, rho)
        
    starthob = random.choice([i for i in range(len(Ngraph))])

    GOAT = [i for i in range(len(Ngraph))]  # initial best ant
    
    i = 0
    while i < runs:
        Ants = []
        #starthob = random.choice([i for i in range(len(Ngraph))])
        
        for j in range(ants):
            stof = FindPath(Pher, Ngraph, starthob, alpha, beta)
            Ants.append(stof)
            GOAT = Elite(GOAT, stof, Ngraph)
            
        # Rank Ants by sorting
        Ranked = Rank(Ants, graph)[:w-1]
        
        #Pheromone update
        for r,j in enumerate(Ranked):
            r += 1
            RBASUpPheromone(j, GOAT, Pher, e, w, r)
            #EASUpPheromone(j, GOAT, Pher, e)
            
        #Pheromone update  
        #EvPheromone(Pher, rho)
        EvPheromoneNeg(Pher, rho)
        
        if i%int(runs*0.05) == 0:
            C = Cost(Ants, Ngraph)
            E = Cost([GOAT], Ngraph)[0]
            print("Evolution ", i, " Elite Cost = ", E,  " Minimum Cost = ", min(C), "  and Maximum Cost = ", max(C))
            #print("<<<<<<<<  The Best Route IS  >>>>>>>> ", GOAT)
        i += 1 

    C = Cost(Ants, Ngraph)    
    print("Minimum Cost Found = ", min(C), "Maximum Cost Found = ", max(C))
    return np.array(Ants), min(C) #, np.array(Pher)

# Example on the Toy Problem

In [101]:
# Toy Network graph 
D = 10
bounds = (2,10)
random.seed(10)
graph = Graph(D, bounds)


In [102]:
# Find Shortest Path with the Algorithm

ants = len(graph)
alpha = 1
beta = 3
rho = 0.1
e = len(graph)
runs = 200
w = int(ants/2)
AS(graph, ants, alpha, beta, rho, e, w, runs)

Evolution  0  Elite Cost =  34  Minimum Cost =  34   and Maximum Cost =  47
Evolution  10  Elite Cost =  33  Minimum Cost =  34   and Maximum Cost =  39
Evolution  20  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  41
Evolution  30  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  37
Evolution  40  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  40
Evolution  50  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  60  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  70  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  80  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  90  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  100  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  110  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  120  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  

(array([[6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 2, 4, 8, 0, 1, 3, 9, 5, 7],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 2, 4, 8, 0, 1, 3, 9, 5, 7],
        [6, 2, 4, 8, 0, 1, 3, 9, 5, 7],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2]]), 33)

# Soving Real TSP problems 

In [115]:
# att48

import RouteMatrix as RM
path = './Data/att48.txt'
graph = RM.TSRM(path)


ants = len(graph)
alpha = 2
beta = 5
rho = 0.01
e = len(graph)
w = int(ants)
runs = 100

AS(graph, ants, alpha, beta, rho, e, w, runs)

Evolution  0  Elite Cost =  36967.64856099629  Minimum Cost =  36967.64856099629   and Maximum Cost =  52878.715681279704
Evolution  5  Elite Cost =  35221.52183745305  Minimum Cost =  35221.52183745305   and Maximum Cost =  40813.07913553732
Evolution  10  Elite Cost =  35221.52183745305  Minimum Cost =  35221.52183745305   and Maximum Cost =  36693.04624410994
Evolution  15  Elite Cost =  35221.52183745305  Minimum Cost =  35221.52183745305   and Maximum Cost =  39158.334619219844
Evolution  20  Elite Cost =  34684.74508635927  Minimum Cost =  35221.52183745305   and Maximum Cost =  36421.50245911037
Evolution  25  Elite Cost =  34684.74508635926  Minimum Cost =  34684.74508635927   and Maximum Cost =  35221.52183745305
Evolution  30  Elite Cost =  34684.74508635926  Minimum Cost =  34684.74508635927   and Maximum Cost =  35842.84432598294
Evolution  35  Elite Cost =  34684.74508635926  Minimum Cost =  35221.52183745305   and Maximum Cost =  37056.22789122697
Evolution  40  Elite Cos

(array([[21, 15,  2, ...,  1, 28, 40],
        [21, 15,  2, ...,  1, 28, 40],
        [21, 15,  2, ...,  1, 28, 40],
        ...,
        [21, 15,  2, ...,  1, 28, 40],
        [21, 15,  2, ...,  1, 28, 40],
        [21, 15,  2, ...,  1, 28, 40]]), 35221.52183745305)

In [116]:
# burma14

import RouteMatrix as RM
path = './Data/burma14.txt'
graph = RM.TSRM(path)


ants = len(graph) 
alpha = 2
beta = 5
rho = 0.01
e = len(graph)
w = int(ants/2)
runs = 1000

AS(graph, ants, alpha, beta, rho, e, w, runs)

Evolution  0  Elite Cost =  36.50430528972049  Minimum Cost =  36.50430528972049   and Maximum Cost =  49.020072414227826
Evolution  50  Elite Cost =  32.176728737294084  Minimum Cost =  35.394607129332805   and Maximum Cost =  43.59428246687915
Evolution  100  Elite Cost =  31.882529491055873  Minimum Cost =  34.726251177601064   and Maximum Cost =  36.50430528972049
Evolution  150  Elite Cost =  31.882529491055873  Minimum Cost =  35.16528840113981   and Maximum Cost =  36.50430528972049
Evolution  200  Elite Cost =  31.882529491055873  Minimum Cost =  35.16528840113981   and Maximum Cost =  36.50430528972049
Evolution  250  Elite Cost =  31.882529491055873  Minimum Cost =  33.70710214662715   and Maximum Cost =  40.595568880377215
Evolution  300  Elite Cost =  31.882529491055873  Minimum Cost =  33.70710214662715   and Maximum Cost =  36.50430528972049
Evolution  350  Elite Cost =  31.882529491055873  Minimum Cost =  33.36667475194933   and Maximum Cost =  35.16528840113981
Evolutio

(array([[ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1],
        [ 2,  3,  4,  5, 11, 13,  6, 12,  7,  0, 10,  8,  9,  1]]),
 32.176728737294084)